In [1]:
# Initialize OpenAI (if Needed)
import os
import json
from openai import OpenAI

client = OpenAI(
    # you will need to register and create an API key
    api_key=os.environ.get('OPENAI_KEY'),
)

## Prose Generation Coding Task

The challenge involves creating a backend system that transforms a series of bulleted "beats" into well-crafted prose that you would find in a novel. This tool is designed to aid writers in fleshing out as scene based on their outlines.

**Part 1: Beats-to-Prose**
Given a set of story beats, like this set for a sci-fi chapter:

1. Begin the chapter with Jack and Xander continuing their excavation on the lunar surface,
creating a sense of tension and anticipation.
2. Describe the barren landscape of the moon, emphasizing its desolation and the isolation
felt by Jack and Xander.
3. Use vivid and descriptive language to portray the moment when Jack and Xander uncover
an alien artifact, highlighting its mysterious and otherworldly appearance.
4. Convey Jack's excitement and trepidation as he realizes the significance of the discovery
and the potential implications it holds for humanity.
5. Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the
urgency of the situation.
6. Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship
and the gravity of the situation.
7. Highlight the importance of the discovery and the potential consequences it could have on
Earth and the lunar mining project.
8. Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a
sense of anticipation and uncertainty.
9. Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges
and dangers that Jack and Xander face.
10. Use sensory details to immerse the reader in the lunar environment, such as the crunch of
lunar soil beneath their boots and the cold, airless atmosphere.
11. Portray Jack's determination and resolve to protect the discovery, even in the face of
potential conflict with rival miners.
12. Hint at the potential dangers and obstacles that Jack and Xander may encounter as they
navigate through a world of corporate greed and betrayal.
13. Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and
anticipation for future chapters.
14. End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to
continue reading and discover what happens next.

Use these beats to produce ~1500 words of prose that matches these beats - in other words, if
a human were to summarize the prose, they would produce something along the lines of these
beats. It's okay to hallucinate details if they are missing from the beats in order to create the
narrative. Target approximately 100-150 words of output per beat. You are allowed to use any LLM or NLP techniques, including open source and closed source services like GPT or Claude.




In [2]:
# Test Input
TEST_BEATS_INPUT = [
    "Begin the chapter with Jack and Xander continuing their excavation on the lunar surface, creating a sense of tension and anticipation.",
    "Describe the barren landscape of the moon, emphasizing its desolation and the isolation felt by Jack and Xander.",
    "Use vivid and descriptive language to portray the moment when Jack and Xander uncover an alien artifact, highlighting its mysterious and otherworldly appearance.",
    "Convey Jack's excitement and trepidation as he realizes the significance of the discovery and the potential implications it holds for humanity.",
    "Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the urgency of the situation.",
    "Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship and the gravity of the situation.",
    "Highlight the importance of the discovery and the potential consequences it could have on Earth and the lunar mining project.",
    "Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a sense of anticipation and uncertainty.",
    "Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges and dangers that Jack and Xander face.",
    "Use sensory details to immerse the reader in the lunar environment, such as the crunch of lunar soil beneath their boots and the cold, airless atmosphere.",
    "Portray Jack's determination and resolve to protect the discovery, even in the face of potential conflict with rival miners.",
    "Hint at the potential dangers and obstacles that Jack and Xander may encounter as they navigate through a world of corporate greed and betrayal.",
    "Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and anticipation for future chapters.",
    "End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to continue reading and discover what happens next."
]

# Write your solution here. Feel free to make methods or classes to do so

## Agentic with context

In [3]:
def chat_with_gpt(messages, max_tokens=400, temperature=0.3):
    """Calls the OpenAI Chat Completion API with the provided messages."""
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )


    cost = completion.usage.prompt_tokens * .5/1e6 + completion.usage.completion_tokens * 1.5/1e6
    return completion.choices[0].message.content.strip(), cost


def context_agent(beat, previous_context=None):
    """
    Analyzes a beat and (optionally) compares it with the previous context.
    Returns a JSON-formatted summary of the scene, e.g.:
    {
        "location": "forest",
        "location_change": false,
        "characters": [
            {"name": "Alice", "character_location": "on stage", "status_change": false},
            {"name": "Bob", "character_location": "off stage", "status_change": false}
        ]
    }
    """
    system_message = (
        "You are ContextAgent. Your job is to extract key scene details from a story beat and compare it with the previous scene context (if provided). "
        "Focus on the following: \n"
        "1. Location: Identify the scene's location. If the new beat describes a location that is essentially the same as the previous context (e.g., the same forest or room), then set 'location_change' to false. Only mark it as true if the beat explicitly indicates a change (for example, 'they enter a cabin' when the previous context was a forest). If no location information is specified, maintain location from the previous beat\n"
        "2. Characters: List each character mentioned in the beat along with their role in the scene. For each character, output their 'character_location' (e.g., 'on stage' if present in the scene, or 'off stage' if only mentioned remotely) and 'status_change': mark false if their role is unchanged from the previous context, or true if the beat indicates a new presence or a different mode of engagement.\n"
        "Return your answer strictly as JSON with keys: 'location', 'location_change', and 'characters' (which is a list of objects with keys 'name', 'character_location', and 'status_change').\n"
        "If no previous context is provided, assume this is the first beat and set all _change flags to false."
    )
    
    # Build the user prompt. If previous_context exists, include it.
    if previous_context:
        user_prompt = (
            f"Previous Context (JSON):\n{previous_context}\n\n"
            f"New Beat:\n\"{beat}\"\n\n"
            "Compare the new beat to the previous context. If the new beat describes the same location, mark 'location_change' as false; "
            "if it indicates a different location, mark it as true and update the 'location'."
            "For each character mentioned, compare with previous context: if their 'character_location' remains the same, mark 'status_change' as false; "
            "if it changes or a new character is introduced, mark it as true. "
            "Return the updated scene context in valid JSON format."
        )
    else:
        user_prompt = (
            f"New Beat:\n\"{beat}\"\n\n"
            "Extract the scene context. Identify the location and any characters along with their involvement (e.g., 'on stage' or 'off stage'). "
            "Return the result in valid JSON format with keys 'location', 'location_change' (false since this is the first beat), and 'characters'."
        )
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
    
    response_text, cost = chat_with_gpt(messages, temperature=0.0)
    
    try:
        # Try to parse the JSON output.
        context_json = json.loads(response_text)
    except Exception as e:
        # If parsing fails, log or handle error appropriately.
        print("Failed to parse context agent output:", response_text)
        context_json = {}
    
    return context_json, cost


In [4]:
cost = 0
for i in range(len(TEST_BEATS_INPUT)):
    
    if i == 0:
        context, c = context_agent(TEST_BEATS_INPUT[i])
    else:
        context, c = context_agent(TEST_BEATS_INPUT[i], context)
        cost += c

    print(f"Context for Beat {i+1}:\n{context}\n")

print(f"Total cost: ${cost:.2f}")

Context for Beat 1:
{'location': 'lunar surface', 'location_change': False, 'characters': [{'name': 'Jack', 'character_location': 'on stage', 'status_change': False}, {'name': 'Xander', 'character_location': 'on stage', 'status_change': False}]}

Context for Beat 2:
{'location': 'lunar surface', 'location_change': False, 'characters': [{'name': 'Jack', 'character_location': 'on stage', 'status_change': False}, {'name': 'Xander', 'character_location': 'on stage', 'status_change': False}]}

Context for Beat 3:
{'location': 'lunar surface', 'location_change': False, 'characters': [{'name': 'Jack', 'character_location': 'on stage', 'status_change': False}, {'name': 'Xander', 'character_location': 'on stage', 'status_change': False}]}

Context for Beat 4:
{'location': 'lunar surface', 'location_change': False, 'characters': [{'name': 'Jack', 'character_location': 'on stage', 'status_change': False}]}

Context for Beat 5:
{'location': 'lunar surface', 'location_change': False, 'characters': 

In [5]:
# ---------------------- Prose Agent ----------------------
def prose_agent(previous_passage, beat_a, beat_b, context_summary, temperature=0.3):
    system_message = (
        "You are ProseAgent, a creative writing assistant specialized in connecting narrative story beats. "
        "Current scene context: " + f"{context_summary}" + ". " +
        "Before writing, think through the key scene details: confirm the location and ensure all characters are actively engaged."
        "Before writing, outline the scene - confirm the previous details of the story and the current story beat, and make sure the story flows normally."
        "For instance, if a communication has ended - don't continue that conversation. Try not to repeat phrases too often, or start sentences with repetitive language."
        "Think about the story setting. What do physics allow? What is the mood and scene?"
        "Do not introduce new plot elements or extraneous details. Your final passage must be between 100 and 150 words."
    )
    
    if previous_passage:
        user_prompt = (
            f"Here is the previous narrative passage:\n\"{previous_passage}\"\n\n"
            f"This is the next story beat:\n\"{beat_b}\"\n\n"
            f"**Current Scene Context:** {context_summary}\n\n"
            "Please think through the scene details and then generate a connecting passage that continues the narrative seamlessly and in a way that makes narrative sense. "
            "Ensure your response is between 100 and 150 words."
        )
    else:
        user_prompt = (
            f"Here are two story beats:\nBeat A: \"{beat_a}\"\nBeat B: \"{beat_b}\"\n\n"
            f"**Current Scene Context:** {context_summary}\n\n"
            "Please think through the scene details first (confirm the location and character engagement), then generate a connecting narrative passage that bridges these beats creatively and seamlessly. "
            "Ensure your response is between 100 and 150 words."
        )
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
    
    return chat_with_gpt(messages, temperature=temperature)

# ---------------------- Story Agent ----------------------
def story_agent(passage, beats):
    """
    Checks whether the newly generated passage is fully consistent with the provided story beats. This means that the passage reflects either BOTH beats or focuses on the second beat.
    The agent must return exactly "True" if the passage faithfully reflects the beats (without extraneous details),
    or "False" if not.
    """
    system_message = (
        "You are StoryAgent, a meticulous narrative consistency checker. Your task is to verify that the given passage "
        "faithfully and exclusively reflects the provided story beats. Do not allow extraneous details or hallucinations. "
        "After reviewing, return ONLY the single word 'True' if the passage is fully consistent, or ONLY 'False' if it is not."
    )
    
    user_prompt = (
        f"Review the following passage:\n\"{passage}\"\n\n"
        "Against these story beats:\n"
    )
    for idx, beat in enumerate(beats, start=1):
        user_prompt += f"Beat {idx}: \"{beat}\"\n"
    user_prompt += "\nReturn ONLY 'True' if the passage is consistent with the beats, otherwise return ONLY 'False'."
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
    
    return chat_with_gpt(messages, temperature=0.0), cost
    

# ---------------------- Length Agent ----------------------
def length_agent(passage, min_words=100, max_words=150):
    """
    Checks whether the newly generated passage meets the desired length.
    Return ONLY "True" if the passage has between min_words and max_words (inclusive),
    otherwise return ONLY "False".
    """
    system_message = (
        "You are LengthAgent, an expert in concise prose evaluation. Your task is to assess whether a given passage meets a specific word count requirement. "
        f"Return ONLY 'True' if the passage contains between {min_words} and {max_words} words (inclusive), or ONLY 'False' if it does not."
    )
    
    word_count = len(passage.split())
    user_prompt = (
        f"Examine the following passage:\n\"{passage}\"\n\n"
        f"It contains {word_count} words. "
        f"Return ONLY 'True' if the word count is between {min_words} and {max_words} inclusive, otherwise return ONLY 'False'."
    )
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    return chat_with_gpt(messages, temperature=0.0), cost




In [6]:
# ---------------------- Multi-Agent Story Generator ----------------------
def generate_story(beats, min_words=100, max_words=150, max_attempts=5):
    """
    Generates a complete story from a list of story beats.
    For each pair of beats:
      1. Use ProseAgent to generate a connecting passage.
      2. Use StoryAgent to verify consistency; if not, retry.
      3. Use LengthAgent to check that the passage meets length requirements; if not, retry.
    The process is recursive for each connecting passage until both checks return "True" or max_attempts is reached.
    
    Arguments:
      - beats: A list of story beat strings.
      - min_words, max_words: The desired word count range for connecting passages.
      - max_attempts: How many times to retry a connecting passage before giving up.
    
    Returns:
      - The final composed story as a string.
    """
    final_story = ""
    current_passage = None
    cost = 0
    # For each pair of beats, generate and validate a connecting passage.
    for i in range(len(beats) - 1):
        if i == 0:
            context, c = context_agent(beats[i])
        else:
            context, c = context_agent(beats[i], context)
        
        cost += c
        beat_a = beats[i]
        beat_b = beats[i + 1]
        
        for attempt in range(max_attempts):
            generated_passage, c = prose_agent(current_passage, beat_a, beat_b, context_summary=context)
            # print(f"ProseAgent output (iteration {i+1}, attempt {attempt+1}):\n{generated_passage}\n")
            cost += c
            # Check consistency
            consistency,c = story_agent(generated_passage, [beat_a, beat_b])
            # print(f"StoryAgent consistency check returned: {consistency}")
            if consistency != "True":
                print("Inconsistency detected; regenerating passage...\n")
                continue  # Rerun prose_agent
            cost += c
            # Check length
            length_ok,c = length_agent(generated_passage, min_words, max_words)
            cost += c
            #
            print(f"LengthAgent check returned: {length_ok}")
            if length_ok != "True":
                print("Length requirement not met; regenerating passage...\n")
                continue  # Rerun prose_agent
            
            # If both checks pass, break out of the retry loop.
            break
        else:
            # If max_attempts were reached without both checks passing, raise an error or accept the last generated passage.
            print(f"Max attempts reached for beats {i+1}. Accepting the last generated passage.")
        
        final_story += f"{generated_passage}\n"
        current_passage = generated_passage  # Use current valid passage for continuity if needed.
    
    # Append the final beat.
    return final_story, cost

In [ ]:
final_story, total_cost = generate_story(TEST_BEATS_INPUT, max_attempts=10)

Inconsistency detected; regenerating passage...



In [ ]:
print(total_cost)
print(final_story)

print(len(final_story.split()))


In [ ]:
completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{'role': 'system', 'content': sys_instruct()},
                {'role': 'user', 'content': (
                    f"Connect these two beats: \"{beats[0]}\" and \"{beats[1]}\". "
                    "Please ensure the connecting prose is between 100 and 150 words."
                )}],
        max_tokens=400,
        temperature=.3
    )
    

In [ ]:

usage = completion.get("usage", {})
tokens_used = usage.get("total_tokens", 0)


In [ ]:
completion.usage.prompt_tokens * .5/1e6 + completion.usage.completion_tokens * 1.5/1e6

### **Part 2: Incorporating Story Metadata**

Now we want to extend the functionality of Beats-to-Prose to accept a few new parameters:
- a list of characters (name and details about them)
- setting
- genre
- style of prose and have that alter the generated prose.

Describe in detail your approach to this. Some things to consider:

1. What models would you use? Why?
2. If you need to procure data, how would go about doing that? How would you prepare the data?
3. What is your approach to producing prose that matches the parameters?

How would you evaluate whether the generations are good quality and fit the parameters specified?
Consider cost and latency considerations

In [13]:
# Write your solution for alterations here. Feel free to make methods or classes to do so

## **Part 3: Describe how you might turn this code into an API**

Imagine you have a working protoype and would like the application team to start using your solution. You can assume you are using the FastAPI framework or something similar. You can use psuedocode for this exercise.

1. What would a hander look like, and what parameters would it receive?

In [ ]:
# Write your solution for the handler here You don't have to run this cell
from fastapi import FastAPI
from fastapi.routing import APIRoute

app = FastAPI()

@app.post("")
async def beat_to_prose_generation():
    return []
